In [3]:
import json
import requests

import pandas as pd
import numpy as np

from tqdm import tqdm 
from geopy.geocoders import Nominatim

import os 
from glob import glob
import warnings
warnings.filterwarnings('ignore')

In [4]:
from haversine import haversine
from geopy.distance import geodesic
import pandas as pd
import numpy as np
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor

In [ ]:
train_path = '../train.csv'
test_path  = '../test.csv'
bus_feature_path = '../bus_feature.csv'
subway_feature_path = '../subway_feature.csv'

df_train = pd.read_csv(train_path)
df_bus = pd.read_csv(bus_feature_path)
df_subway = pd.read_csv(subway_feature_path)
df_test = pd.read_csv(test_path)

In [4]:
df_bus = df_bus.rename({'Y좌표' : '좌표Y', 'X좌표' : '좌표X'}, axis=1)

# utils

In [5]:
geo_local = Nominatim(user_agent='South Korea')

def geocoding(address):
    try:
        geo = geo_local.geocode(address)
        if geo:
            return geo.longitude, geo.latitude
        else:
            return np.nan, np.nan
    except:
        return np.nan, np.nan

In [6]:
def calculate_distances_vectorized(row_train, df_loc, distance, alpha=0):
    coords_row = np.radians(np.array([row_train[0], row_train[1]]))
    coords_df = np.radians(df_loc[['좌표Y', '좌표X']].values.T)
    distances = np.linalg.norm(coords_row[:, np.newaxis] - coords_df, axis=0) * 6371000  # 차원을 확장하여 브로드캐스팅
    return np.sum((distances + alpha <= distance).astype(int))

In [7]:
def count_number_of_object(df_main, df_object, distances, alpha=0):
    df_main_loc = df_main[['좌표Y', '좌표X']]
    df_object_loc = df_object[['좌표Y', '좌표X']]

    with ThreadPoolExecutor() as executor:
        result = list(executor.map(lambda row_train: calculate_distances_vectorized(row_train, df_object_loc, distances),
                                    tqdm(df_main_loc.itertuples(index=False),
                                        total=len(df_main_loc), desc='Building Iteration', position=1)))

    df = pd.DataFrame(result)
    return df

In [8]:
def geocode_addresses(address_chunk):
    return address_chunk.apply(geocoding)

def parallel_geocoding(address_list, chunk_size=1000):
    address_chunks = [address_list[i:i + chunk_size] for i in range(0, len(address_list), chunk_size)]

    with ProcessPoolExecutor() as executor:
        results = list(executor.map(geocode_addresses, address_chunks))

    return pd.concat(results)

In [9]:
def geocoding_kakao(address):
    api_url = "https://dapi.kakao.com/v2/local/search/address.json"
    headers = {"Authorization": f"KakaoAK {'04e8964d98b590c7f9fb298d4071f292'}"}
    params = {"query": address}
    try:
        response = requests.get(api_url, headers=headers, params=params)
        response.raise_for_status()
        result = response.json()

        if "documents" in result and len(result["documents"]) > 0:
            return result["documents"][0]["y"], result["documents"][0]["x"]
        else:
            return None
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None

# geopy라이브러리를 이용해 도로명주소로 위도 경도 구하기 (좌표X, 좌표Y 채우기)

In [10]:
# 시군으로 나누기
df_train['시군'] = df_train['시군구'].str.split().str[:2].apply(lambda x: ' '.join(x))
df_test['시군'] = df_test['시군구'].str.split().str[:2].apply(lambda x: ' '.join(x))

# 도로명 데이터를 이용하여 도로명주소 생성
df_train['주소'] = df_train['시군구'] + ' ' + df_train['번지']
df_train['도로명주소'] = df_train['시군'] + ' ' + df_train['도로명']

df_test['주소'] = df_test['시군구'] + ' ' + df_test['번지']
df_test['도로명주소'] = df_test['시군'] + ' ' + df_test['도로명']

# 번지 주소가 없다면 도로명 주소로 채워줌, 번지 주소는 loc을 위함 
df_train.loc[df_train['주소'].isna(), '주소'] = df_train.loc[df_train['주소'].isna(), '시군'] + ' ' + df_train.loc[df_train['주소'].isna(), '도로명']
df_test.loc[df_test['주소'].isna(), '주소'] = df_test.loc[df_test['주소'].isna(), '시군'] + ' ' + df_test.loc[df_test['주소'].isna(), '도로명']

df_train.drop([1050833, 1113986], inplace=True)

In [11]:
train_address = df_train[(df_train['좌표X'].isna()) | (df_train['좌표Y'].isna())]['주소'].drop_duplicates()
test_address = df_test[(df_test['좌표X'].isna()) | (df_test['좌표Y'].isna())]['주소'].drop_duplicates()

In [1]:
loc_train = parallel_geocoding(train_address)
loc_test = parallel_geocoding(test_address)

loc_train = loc_train.apply(pd.Series)
loc_test = loc_test.apply(pd.Series)

loc_train.columns = ["좌표X", "좌표Y"]
loc_test.columns = ["좌표X", "좌표Y"]

loc_train = pd.concat([train_address, loc_train], axis=1)
loc_test = pd.concat([test_address, loc_test], axis=1)

In [437]:
nan_address_train = loc_train[loc_train['좌표X'].isna()]['주소']
nan_address_train_df = df_train.iloc[nan_address_train.index]
new_address_train = nan_address_train_df['시군'] + ' ' + nan_address_train_df['도로명']

nan_loc_train = new_address_train.apply(geocoding)

nan_loc_train = nan_loc_train.apply(pd.Series)
nan_loc_train.columns = ["좌표X", "좌표Y"]
loc_train.loc[loc_train['좌표X'].isna(), ['좌표X', '좌표Y']] = nan_loc_train

In [ ]:
nan_address_train_2 = loc_train[loc_train['좌표X'].isna()]['주소']

nan_loc_train_2 = new_address_train_2.apply(geocoding_kakao)

nan_loc_train_2 = nan_loc_train_2.apply(pd.Series)
nan_loc_train_2.columns = ["좌표X", "좌표Y"]
loc_train.loc[loc_train['좌표X'].isna(), ['좌표X', '좌표Y']] = nan_loc_train_2

In [438]:
nan_address_test = loc_test[loc_test['좌표X'].isna()]['주소']
nan_address_test_df = df_test.iloc[nan_address_test.index]
new_address_test = nan_address_test_df['시군'] + ' ' + nan_address_test_df['도로명']

nan_loc_test = new_address_test.apply(geocoding)

nan_loc_test = nan_loc_test.apply(pd.Series)
nan_loc_test.columns = ["좌표X", "좌표Y"]
loc_test.loc[loc_test['좌표X'].isna(), ['좌표X', '좌표Y']] = nan_loc_test

In [16]:
loc_train.to_csv('../loc_train.csv', index=False)
loc_test.to_csv('../loc_test.csv', index=False)

In [14]:
loc_train = pd.read_csv('../loc_train.csv')
loc_test = pd.read_csv('../loc_test.csv')

In [18]:
loc_train.isna().sum(), loc_test.isna().sum()

(주소     0
 좌표X    0
 좌표Y    0
 dtype: int64,
 주소     0
 좌표X    0
 좌표Y    0
 dtype: int64)

In [19]:
for data in tqdm(loc_train.iterrows(), desc="Processing"):
    df_train.loc[df_train['주소'] == data[1][0], ["좌표X", "좌표Y"]] = df_train.loc[df_train['주소'] == data[1][0], ["좌표X", "좌표Y"]].fillna({"좌표X" : data[1][1], "좌표Y" : data[1][2]})

Processing: 8193it [13:24, 10.19it/s]


In [20]:
for data in tqdm(loc_test.iterrows(), desc="Processing"):
    df_test.loc[df_test['주소'] == data[1][0], ["좌표X", "좌표Y"]] = df_test.loc[df_test['주소'] == data[1][0], ["좌표X", "좌표Y"]].fillna({"좌표X" : data[1][1], "좌표Y" : data[1][2]})

Processing: 2015it [00:04, 448.70it/s]


In [21]:
df_train['좌표X'].isna().sum(), df_test['좌표X'].isna().sum()

(0, 0)

In [28]:
df_train.to_csv('../fill_loc_train.csv', index=False)

In [24]:
df_test.to_csv('../fill_loc_test.csv', index=False)

# 버스정류장 개수 데이터프레임


In [29]:
bus_count_train_df = count_number_of_object(df_train, df_bus, 500)
bus_count_train_df.columns = ['500m이내 정류장개수']

Building Iteration: 100%|██████████| 1118820/1118820 [13:35<00:00, 1372.76it/s]


In [31]:
bus_count_train_df.to_csv('../bus_count_train.csv', index=False)

In [22]:
bus_count_test_df = count_number_of_object(df_test, df_bus, 500)
bus_count_test_df.columns = ['500m이내 정류장개수']

Building Iteration: 100%|██████████| 9272/9272 [00:04<00:00, 2311.53it/s]


In [ ]:
bus_count_test_df.to_csv('../bus_count_test.csv', index=False)

# 지하철역 개수 데이터프레임

In [32]:
df_subway = df_subway.rename({'위도' : '좌표Y', '경도' : '좌표X'}, axis=1)
df_subway

,역사_ID,역사명,호선,좌표Y,좌표X
0,9996,미사,5호선,37.560927,127.193877
1,9995,강일,5호선,37.557490,127.175930
2,4929,김포공항,김포골드라인,37.562360,126.801868
3,4928,고촌,김포골드라인,37.601243,126.770345
4,4927,풍무,김포골드라인,37.612488,126.732387
...,...,...,...,...,...
763,154,종로5가,1호선,37.570926,127.001849
764,153,종로3가,1호선,37.570406,126.991847
765,152,종각,1호선,37.570161,126.982923
766,151,시청,1호선,37.565715,126.977088


In [33]:
subway_count_train_df = count_number_of_object(df_train, df_subway, 500)
subway_count_train_df.columns = ['500m이내 지하철역 개수']

Building Iteration: 100%|██████████| 1118820/1118820 [12:29<00:00, 1493.50it/s]


In [38]:
subway_count_train_df.to_csv('../subway_count_train.csv', index=False)

In [25]:
subway_count_test_df = count_number_of_object(df_test, df_subway, 500)
subway_count_test_df.columns = ['500m이내 지하철역 개수']

Building Iteration: 100%|██████████| 9272/9272 [00:05<00:00, 1814.98it/s]


In [ ]:
subway_count_test_df.to_csv('../subway_count_test.csv', index=False)

# 학교 개수 데이터프레임

In [39]:
df_school_info = pd.read_csv('../seoul_school_info.csv', encoding='cp949')
df_school_info

,학교종류명,설립구분,표준학교코드,학교명,영문학교명,관할조직명,도로명우편번호,도로명주소,도로명상세주소,전화번호,...,주야구분명,설립일자,개교기념일,시도교육청코드,시도교육청명,소재지명,주야과정,계열명,학과명,적재일시
0,각종학교(중),사립,7134155,선화예술중학교,Sunhwa Arts Middle School,서울특별시성동광진교육지원청,4991,서울특별시 광진구 천호대로 664,/ 선화예술중고등학교 (능동),02-2204-1100,...,주간,19731201,19730705,B10,서울특별시교육청,서울특별시,NaN,NaN,NaN,20230627
1,초등학교,공립,7134150,서울숭신초등학교,Seoul Soongshin Elementary School,서울특별시성동광진교육지원청,4702,서울특별시 성동구 마장로 161,(하왕십리동),02-2252-5950,...,주간,19590403,19590403,B10,서울특별시교육청,서울특별시,NaN,NaN,NaN,20230615
2,중학교,공립,7134142,행당중학교,Haengdang Middle School,서울특별시성동광진교육지원청,4764,서울특별시 성동구 왕십리로 189,(행당동/행당중학교),02-2292-2721,...,주간,19680806,19681002,B10,서울특별시교육청,서울특별시,NaN,NaN,NaN,20230615
3,중학교,사립,7134141,한양대학교사범대학부속중학교,Hanyang University Middle School,서울특별시성동광진교육지원청,4761,서울특별시 성동구 마조로 42,(사근동/한양사대부속중?고등학교),02-2200-3700,...,주간,19600118,19600118,B10,서울특별시교육청,서울특별시,NaN,NaN,NaN,20230615
4,중학교,공립,7134140,자양중학교,Jayang Middle School,서울특별시성동광진교육지원청,5069,서울특별시 광진구 뚝섬로41길 33,(자양동/서울자양중학교),02-446-0365,...,주간,19840301,19840301,B10,서울특별시교육청,서울특별시,NaN,NaN,NaN,20230615
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3927,고등학교,국립,1371661,국립국악고등학교,Gugak National High School,교육부,6311,서울특별시 강남구 개포로22길 65,(개포동/ 국악고등학교),02-3460-0500,...,주간,19550401,19720909,B10,서울특별시교육청,서울특별시,주간,예술계,무용과,20230615
3928,특수학교,국립,1342102,한국우진학교,Hanguk Woojin School,교육부,3934,서울특별시 마포구 월드컵북로38길 21,/ 한국우진학교 (중동),02-6388-5800,...,주간,20000301,20000306,B10,서울특별시교육청,서울특별시,NaN,NaN,NaN,20230709
3929,특수학교,국립,1342099,서울농학교,Seoul National school for the Deaf,교육부,3032,서울특별시 종로구 필운대로 103,(신교동),02-737-0659,...,주간,19130401,19130401,B10,서울특별시교육청,서울특별시,NaN,NaN,NaN,20230709
3930,특수학교,국립,1342098,서울맹학교,Seoul National School for the Blind,교육부,3032,서울특별시 종로구 필운대로 97,(신교동),02-731-6772,...,주간,19130401,19130401,B10,서울특별시교육청,서울특별시,NaN,NaN,NaN,20230709


In [40]:
school_address = df_school_info['도로명주소'].drop_duplicates()
school_address

0          서울특별시 광진구 천호대로 664
1           서울특별시 성동구 마장로 161
2          서울특별시 성동구 왕십리로 189
3            서울특별시 성동구 마조로 42
4         서울특별시 광진구 뚝섬로41길 33
                ...          
3914     서울특별시 금천구 시흥대로38길 62
3925      서울특별시 강남구 개포로22길 65
3928    서울특별시 마포구 월드컵북로38길 21
3929       서울특별시 종로구 필운대로 103
3930        서울특별시 종로구 필운대로 97
Name: 도로명주소, Length: 1238, dtype: object

In [41]:
school_loc = school_address.apply(geocoding)

In [42]:
school_loc = school_loc.apply(pd.Series)
school_loc.columns = ["좌표X", "좌표Y"]
df_school_loc = pd.concat([school_address, school_loc], axis=1)
df_school_loc

,도로명주소,좌표X,좌표Y
0,서울특별시 광진구 천호대로 664,127.073812,37.559291
1,서울특별시 성동구 마장로 161,127.039174,37.565757
2,서울특별시 성동구 왕십리로 189,127.041541,37.557618
3,서울특별시 성동구 마조로 42,127.041832,37.561366
4,서울특별시 광진구 뚝섬로41길 33,127.075387,37.534953
...,...,...,...
3914,서울특별시 금천구 시흥대로38길 62,126.906917,37.445781
3925,서울특별시 강남구 개포로22길 65,127.051106,37.475738
3928,서울특별시 마포구 월드컵북로38길 21,126.904304,37.571304
3929,서울특별시 종로구 필운대로 103,126.968707,37.584827


In [43]:
school_count_train_df = count_number_of_object(df_train, df_school_loc, 500)
school_count_train_df.columns = ['500m이내 학교 개수']

Building Iteration: 100%|██████████| 1118820/1118820 [12:40<00:00, 1470.99it/s]


In [44]:
school_count_train_df.to_csv('../school_count_train.csv', index=False)

In [36]:
school_count_test_df = count_number_of_object(df_test, df_school_loc, 500)
school_count_test_df.columns = ['500m이내 학교 개수']

Building Iteration: 100%|██████████| 9272/9272 [00:02<00:00, 3379.99it/s]


In [192]:
school_count_test_df.to_csv('../school_count_test.csv', index=False)

# 종합병원 개수 데이터프레임

In [45]:
df_hospital_info = pd.read_csv('../seoul_hospital_info.csv', encoding='cp949')
df_hospital_info.rename({'병원경도' : '좌표X', '병원위도' : '좌표Y'}, axis=1, inplace=True)
df_hospital_info

,기관ID,주소,병원분류,병원분류명,응급의료기관코드,응급의료기관코드명,응급실운영여부(1/2),비고,기관설명상세,간이약도,...,진료시간(목요일)S,진료시간(금요일)S,진료시간(토요일)S,진료시간(일요일)S,진료시간(공휴일)S,우편번호1,우편번호2,좌표X,좌표Y,작업시간
0,A1120837,"서울특별시 금천구 벚꽃로 286, 삼성리더스타워 111~114호 (가산동)",C,의원,G099,응급의료기관 이외,2,"외과: 상시진료, 내과는 당분간 휴진","서울시 송파구 문정동 장지동 법조단지 위례 가락동 가락시장역 위치, 삼성서울병원 외...",방이역 1번출구 바로옆 굿모닝 신한증권 뒷건물,...,900,900,900,1000.0,1000.0,85,11,126.884122,37.480394,2022-04-07 14:55:00.0
1,A1104130,"서울특별시 강남구 도산대로 118 (논현동, 신사빌딩 2층)",C,의원,G099,응급의료기관 이외,2,비만클리닉,"365일 진료, 국민건강보험공단 검진 지정병원 점심시간 13~14시",신사역 1번 출구 인근,...,1100,1100,1100,900.0,900.0,60,38,127.021809,37.516604,2021-06-07 14:54:49.0
2,A1124291,서울특별시 강남구 봉은사로 612 (삼성동),B,병원,G099,응급의료기관 이외,2,"인공신장실: 월,수,급 야간진료",NaN,성신여대입구역 7번출구/ 골든타워 5~6층,...,900,900,900,900.0,NaN,61,70,127.062147,37.514281,2024-01-07 14:56:00.0
3,A1117873,"서울특별시 관악구 신원로 38, 5층 (신림동, 청암빌딩)",N,치과의원,G099,응급의료기관 이외,2,대표번호1 지역번호 추가20170118150453,"서울시 송파구 문정동 장지동 법조단지 위례 가락동 가락시장역 위치, 삼성서울병원 외...",서월치안센터 인근 청암빌딩 5층,...,1400,1000,1000,1000.0,1000.0,87,76,126.929377,37.481918,2022-01-07 14:54:55.0
4,B1100027,서울특별시 강남구 봉은사로 612 (삼성동),E,한방병원,G099,응급의료기관 이외,2,진료과목마다 진료시간 다릅니다. 전화 확인 후 방문해주세요.,NaN,봉은사역 5번 출구 삼성1동주민센터 옆,...,900,900,900,900.0,NaN,61,70,127.062147,37.514281,2024-01-07 14:56:00.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20312,A1100240,"서울특별시 양천구 신목로 120 (목동, 힘찬병원)",B,병원,G099,응급의료기관 이외,2,점심시간 13:00~14:00,"눈의 편안함을 약속하는 힐링안과입니다. 힐링안과는 재수술 센터 보유하고 있으며,국내...",오목교역 4번 출구,...,900,900,900,900.0,930.0,79,99,126.876845,37.524983,2024-01-07 14:56:30.0
20313,B1101664,"서울특별시 용산구 한강대로 109 (한강로2가, 용성비즈텔)",G,한의원,G099,응급의료기관 이외,2,점심시간 13:00~14:00,"눈의 편안함을 약속하는 힐링안과입니다. 힐링안과는 재수술 센터 보유하고 있으며,국내...","신용산역 5,6번 출구",...,930,930,930,900.0,930.0,43,76,126.968173,37.529991,2024-01-07 14:56:30.0
20314,C1108476,"서울특별시 강남구 봉은사로 213, 센트럴타워 5~7층 (논현동)",E,한방병원,G099,응급의료기관 이외,2,점심시간 13:00~14:00,"눈의 편안함을 약속하는 힐링안과입니다. 힐링안과는 재수술 센터 보유하고 있으며,국내...","신용산역 5,6번 출구",...,900,900,900,900.0,930.0,61,9,127.035366,37.508050,2024-01-07 14:56:30.0
20315,C1108345,"서울특별시 강서구 강서로 173, 터보빌딩 7,8,9층 (화곡동)",E,한방병원,G099,응급의료기관 이외,2,점심시간 13:00~14:00,"눈의 편안함을 약속하는 힐링안과입니다. 힐링안과는 재수술 센터 보유하고 있으며,국내...","신용산역 5,6번 출구",...,900,900,900,900.0,930.0,77,14,126.839622,37.541922,2024-01-07 14:56:30.0


In [46]:
df_hospital_info = df_hospital_info[df_hospital_info['병원분류명'] == '종합병원']

In [47]:
hospital_count_train_df = count_number_of_object(df_train, df_hospital_info, 5000, 1500)
hospital_count_train_df.columns = ['5km이내 종합병원 개수']

Building Iteration: 100%|██████████| 1118820/1118820 [06:59<00:00, 2669.33it/s]


In [48]:
hospital_count_train_df.to_csv('../hospital_count_train.csv', index=False)

In [41]:
hospital_count_test_df = count_number_of_object(df_test, df_hospital_info, 5000, 1500)
hospital_count_test_df.columns = ['5km이내 종합병원 개수']

Building Iteration: 100%|██████████| 9272/9272 [00:04<00:00, 2271.61it/s]


In [412]:
hospital_count_test_df.to_csv('../hospital_count_test.csv', index=False)

5km이내 종합병원개수
2               288340
3               276988
4               215539
1               103470
5               102317
6                66084
0                21107
9                17800
8                13538
10                7215
7                 6422
dtype: int64

# 공원 개수 데이터프레임

In [49]:
df_park_info = pd.read_csv('../seoul_park.csv', encoding='cp949')
df_park_info.rename({'X좌표(WGS84)' : '좌표X', 'Y좌표(WGS84)' : '좌표Y'}, axis=1, inplace=True)
df_park_info

,연번,공원명,공원개요,면적,개원일,주요시설,주요식물,안내도,오시는길,이용시참고사항,이미지,지역,공원주소,관리부서,전화번호,X좌표(GRS80TM),Y좌표(GRS80TM),좌표X,좌표Y,바로가기
0,1,남산도시자연공원,남산공원은 도심에 위치하여 서울시민에게 맑은 공기를 제공하는 자연휴식처이며 산책 꽃...,2896887㎡ 임 야 : 2454140㎡ 녹지대 및 기타시설 : 442747㎡,1968.9.10,기반시설 : 광장 45950㎡ 도로 108530㎡ 산책로 6.7㎞ (북측:3.7㎞/...,소나무 단풍 아카시아 상수리나무 등 191종 2881870주,http://parks.seoul.go.kr/upload/seditorMulti/2...,명동역(남산케이블카 와룡묘 서울애니메이션센터 방면) 도보 명동역 3번출구 → 퍼시픽...,남산공원 차량통행 금지안내 2005년 5월 1일부터 남산공원에 일반승용차 택시 통행...,http://parks.seoul.go.kr/file/info/view.do?fId...,중구,서울특별시 중구 삼일대로 231(예장동),서울특별시중부푸른도시사업소,02-3783-5900,198364.107000,4.503956e+05,126.990377,37.550140,http://parks.seoul.go.kr/template/sub/namsan.do
1,2,길동생태공원,길동생태공원은 생물의 서식처를 제공하고 종다양성을 증진시키며 자연생태계의 생물들을 ...,80683㎡,1999.5.20,탐방객안내소 야외전시대 관찰대 목재데크 조류관찰대 길동생태문화센터,소나무 보리수 등 64종 31800주 산국 부들 등 138종 192800본,http://parks.seoul.go.kr/template/common/img/p...,NaN,매주 월요일은 휴관입니다 사전예약 후 입장하실 수 있습니다. ▶ 인터넷 예약 바로가...,http://parks.seoul.go.kr/file/info/view.do?fId...,강동구,서울특별시 강동구 천호대로 1291(길동생태공원),동부공원녹지사업소 길동생태공원,02-489-2770,213554.120000,4.488527e+05,127.154779,37.540394,http://parks.seoul.go.kr/template/sub/gildong.do
2,3,서울대공원,서울대공원은 세계 각국의 야생동물들이 살아 숨 쉬는 서울동물원과 다양한 재미와 즐거...,9132690m²,1984.5.1,동물원 식물원 테마가든(장미원 어린이동물원 피크닉장) 치유숲 산림욕장 캠핑장 국립현...,NaN,NaN,지하철 4호선 대공원역 하차(2번 출구) 도보 15분 정도 문의처 02)500-73...,일반사항 공원을 깨끗하게 이용합니다. 대중교통을 이용해 주세요. 기념물 시설물 풀과...,http://parks.seoul.go.kr/file/info/view.do?fId...,과천시,경기도 과천시 대공원광장로 102,전략기획실,02-500-7032,200994.267000,4.371640e+05,127.019846,37.426449,http://grandpark.seoul.go.kr/
3,4,서울숲,당초 골프장 승마장 등이 있던 뚝섬일대를 주거업무 지역으로 개발할 경우 약 4조원에...,480994㎡,2005.6.18,4개 테마공원: 문화예술공원 자연생태숲 자연체험학습원 습지생태원 주요시설: 가족마당...,수 목 : 소나무 섬잣나무 계수나무 외 95종 415795주 식물원 : 선인장 등 ...,http://parks.seoul.go.kr/template/common/img/p...,NaN,지하철 분당선 서울숲역 3번출구 도보로 약5분 2호선 뚝섬역 8번출구 도보로 약 1...,http://parks.seoul.go.kr/file/info/view.do?fId...,성동구,서울특별시 성동구 뚝섬로 273 (성수동1가),서울숲 관리사무소,02-460-2905,203695.432000,4.492907e+05,127.041798,37.543072,http://parks.seoul.go.kr/template/sub/seoulfor...
4,5,월드컵공원,월드컵공원은 서울의 서쪽에 위치하여 1978년부터 1993년까지 15년간 서울시민이...,2284085㎡,2002.5.1,평화의 공원 월드컵공원전시관(879㎡) 유니세프광장(2400㎡) 평화광장(5217㎡...,NaN,http://parks.seoul.go.kr/template/common/img/p...,지하철 6호선 월드컵경기장역 하차 → 1번 출구로 나온 후 직진 → 큰길(도로)이 ...,일반사항 공원을 깨끗하게 이용합니다. 대중교통을 이용해 주세요. 기념물 시설물 풀과...,http://parks.seoul.go.kr/file/info/view.do?fId...,마포구,서울특별시 마포구 하늘공원로 84(월드컵공원),월드컵공원,02-300-5500,190658.070000,4.515988e+05,126.878907,37.571805,http://parks.seoul.go.kr/template/sub/worldcup...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,128,용마도시자연공원(사가정공원),2005년 4월 13일 개장한 사가정공원은 면목동 산 50번지 일대의 면목약수터지구...,109635㎡,2005.4.13,피크닉장 놀이터 체력단련장 체육관,NaN,NaN,NaN,NaN,http://parks.seoul.go.kr/file/info/view.do?fId...,중랑구,서울특별시 중랑구 면목7동 산50,중랑구청 공원녹지과,02-2094-2993,NaN,NaN,127.095997,37.579762,NaN
128,129,문화비축기지,매봉산자락에 위치한 문화비축기지는 폐산업시설인 '마포석유비축기지'를 재생하여 역사적...,140022㎡,2017.9.1,6개의탱크와 문화마당 녹지,NaN,http://parks.seoul.go.kr/upload/seditorMulti/2...,NaN,? 일반사항 - 공원은 연중무휴이나 전시관은 월요일 휴관입니다. - 공원을 ...,http://parks.seoul.go.kr/file/info/view.do?fId...,마포구,서울특별시 마포구 증산로 87,문화비축기지,02-376-8410,114264.460400,3.712093e+06,126.893246,37.571718,http://parks.seoul.go.kr/template/sub/culturet...
129,130,경춘선숲길,일제 강점기때 우리 민족의 자본으로 만든 최초의 철도시설로 1939년 7월 25일 ...,211392㎡,NaN,조경시설 : 텃밭 잔디광장 철길 산책로 등 건축시설 : 무궁화호 객차(방문자센터) ...,NaN,NaN,지하철 이용시 지하철 7호선 하계역 4번출구 미성/장미아파트 도로 450m직진 지하...,○ 6.3km (시 작 점 : 육사삼거리 중간지점 : 화랑대사거리 → 화랑대...,http://parks.seoul.go.kr/file/info/view.do?fId...,NaN,노원구 공릉2동 산 82-2,중부공원녹지사업소,02-2289-4012,127.076482,3.762708e+01,127.076482,37.627077,NaN
130,131,율현공원,? 율현공원은 보금자리주택 세곡2지구 사업의 결과물로 만들어졌다. 공원은 주택지구의...,157535㎡,2016.11.1,관리시설 : 사무실 조경시설 : 정자(4개) 조형쉼터(10개) 조형파고라(3개) 수...,수목 : 소나무 이팝나무 등 110종 143069주 초화 : 맥문동 벌개미취 등...,http://parks.seoul.go.kr/upload/seditorMulti/2...,NaN,공원 인근에 서울공항 활주로가 위치하고 있어 공원 내에서 드론을 날리거나 헬륨가스 ...,http://parks.seoul.go.kr/file/info/view.do?fId...,강남구,서울특별시 강남구 율현동 밤고개로24길(율현동 56),동부공원녹지사업소,02-459-9452,NaN,NaN,127.1155

In [50]:
park_count_train_df = count_number_of_object(df_train, df_park_info, 1500)
park_count_train_df.columns = ['1.5km이내 공원 개수']

Building Iteration: 100%|██████████| 1118820/1118820 [06:59<00:00, 2664.86it/s]


In [51]:
park_count_train_df.to_csv('../park_count_train.csv', index=False)

In [47]:
park_count_test_df = count_number_of_object(df_test, df_park_info, 1500)
park_count_test_df.columns = ['1.5km이내 공원 개수']

Building Iteration: 100%|██████████| 9272/9272 [00:03<00:00, 2427.37it/s]


In [365]:
park_count_test_df.to_csv('../park_count_test.csv', index=False)